## Prepare

In [1]:
import os
import requests
import pandas as pd

from tqdm.auto import tqdm
s = requests.session()

P1_API_TOKEN = os.environ["P1_EDGAR_API_TOKEN"]

s.headers.update({'Authorization': f'Token {P1_API_TOKEN}'})

## Function for getting headers

In [2]:
def get_headers(**kwargs) -> pd.DataFrame:
    """Get the headers data with the certain parameters"""
    data = []
    with tqdm() as pbar:
        r = s.get(f'https://data.particle.one/edgar/v5/data/headers',
                  params=kwargs)
        while True:
            d = r.json()
            if not pbar.total:
                pbar.reset(total=d['count'])
            _ = pbar.update(len(d['data']))
            data.extend(d['data']);
            nxt = d.get('links', {}).get('next')
            if not nxt:
                break
            r = s.get(nxt) 
    return pd.DataFrame(data)

## Get the headers data first time. Find out the last date and time.

In [3]:
df = get_headers(start_date='2021-02-10', date_mode='knowledge_date')

0it [00:00, ?it/s]

In [4]:
last_timestamp = df["created_at"].max()

In [5]:
last_timestamp

'2021-02-10T16:07:06.256911+00:00'

## Get the headers data second time. Get the difference since last date and time

In [9]:
# Wait some time to allow the pipline to get more data.
df = get_headers(start_date='2021-02-10', date_mode='knowledge_date')

0it [00:00, ?it/s]

In [10]:
new_data = df[df["created_at"] > last_timestamp]

In [11]:
new_data

,uuid,cik,filing_date,form_type,edgar_path,company,acceptance_datetime,period_of_report,created_at
612,dbe1b28b-a9f9-4ab5-9d59-2743f92ec426,84748,2021-02-10,4,edgar/data/84748/000166427221000017/0001664272...,ROGERS CORP,2021-02-10T15:58:26+00:00,2021-02-08T05:00:00+00:00,2021-02-10T16:10:07.172266+00:00
681,8bbafa79-b14a-41f2-aeb5-5e09831c5492,1058800,2021-02-10,13F-HR,edgar/data/1058800/000105880021000001/00010588...,BEACH INVESTMENT COUNSEL INC/PA,None,None,2021-02-10T16:10:55.674297+00:00
789,6c916526-05dd-4c7f-b010-b43fdadac4c1,1420485,2021-02-10,4,edgar/data/1420485/000166427221000017/00016642...,Weaver Mark,2021-02-10T15:58:26+00:00,2021-02-08T05:00:00+00:00,2021-02-10T16:10:09.293633+00:00
970,488b33e2-0cc5-49ee-b0cd-af03fbeb9dd1,1799054,2021-02-10,13F-HR,edgar/data/1799054/000142050621000166/00014205...,"Silicon Hills Wealth Management, LLC",2021-02-10T15:57:28+00:00,2020-12-31T05:00:00+00:00,2021-02-10T16:10:05.164739+00:00
